<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-ssungz789/blob/main/w5/%EC%88%98%EB%8A%A5_of_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습: GPT로 prompting 기법들 체험하기

이번 실습에서는 GPT api를 통해 이론 시간에 배운 prompting 기법들을 다룹니다. 먼저 필요한 library들을 설치합니다.

In [1]:
!pip -q install openai datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [2]:
import json
from google.colab import drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/MyDrive/colab_sparta/2023_11_KICS.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

그 다음 openai api key를 다음과 같은 절차를 거쳐 얻어냅니다:
1. platform.openai.com 에 계정을 생성하여 로그인합니다.
2. `Dashboard > API keys` 메뉴로 들어가 `+ Create new secret key`를 눌러줍니다.
3. 이름을 작성한 후, `Create secret key`를 눌러 key를 만들어줍니다.
4. 생성된 key를 복사한 후 아래 "OPENAI_API_KEY"에 불여넣어줍니다.

In [5]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [6]:
def prediction(problem, paragraph):
    # 문제와 보기를 하나의 문자열로 결합
    prompt = f"다음은 국어 수능 문제다.\n\n지문:\n{paragraph}\n\n문제: {problem['question']}\n\n보기:\n"
    for i, choice in enumerate(problem['choices']):
        prompt += f"{i+1}. {choice}\n"
    prompt += "\n위 문제의 정답으로 가장 적절한 것의 번호를 선택하고, 답변은 숫자로만 해주세요."

    # GPT-4에 요청
    response = client.chat.completions.create(
        model="gpt-4",
        n=1,
        messages=[
            {"role": "system", "content": "너는 한국어를 국어 선생님 만큼 엄청 잘하는 AI야. 주어진 수능 문제를 해결하고 가장 적절한 답을 선택해."},
            {"role": "user", "content": prompt}
        ]
    )

    # GPT-4의 응답에서 답변 추출
    answer = response.choices[0].message.content

    # 답변에서 숫자만 추출
    import re
    numbers = re.findall(r'\d+', answer)
    if numbers:
        return int(numbers[0]) - 1  # 0-based index로 변환
    else:
        return -1  # 유효한 답변을 찾지 못한 경우

In [7]:
from tqdm import tqdm

total_score = 0
earned_score = 0

for item in tqdm(data):
    paragraph = item.get('paragraph', '')  # paragraph가 없는 경우를 대비해 기본값 설정
    for problem in item['problems']:
        predicted_answer = prediction(problem, paragraph)
        correct_answer = problem['answer'] - 1  # 0-based index로 변환
        print(f"문제: {problem['question']}")
        print(f"정답: {problem['answer']}")
        print(f"GPT-4의 예측: {predicted_answer + 1}")

        if predicted_answer == correct_answer:
            earned_score += problem['score']

        total_score += problem['score']



  0%|          | 0/11 [00:00<?, ?it/s]

문제: 윗글의 내용과 일치하지 않는 것은?
정답: 4
GPT-4의 예측: 4
문제: 다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은?
정답: 5
GPT-4의 예측: 3


  9%|▉         | 1/11 [00:01<00:15,  1.55s/it]

문제: 윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은?
정답: 1
GPT-4의 예측: 1
문제: (가)와 (나)에 대한 설명으로 가장 적절한 것은?
정답: 4
GPT-4의 예측: 4
문제: [A]에 대한 이해로 적절하지 않은 것은?
정답: 5
GPT-4의 예측: 5
문제: ㉮에 대한 이해를 바탕으로 ㉠, ㉡에 대해 파악한 내용으로 적절하지 않은 것은?
정답: 3
GPT-4의 예측: 3
문제: ㉯를 반박하기 위한 ‘이수광’의 말로 가장 적절한 것은?
정답: 2
GPT-4의 예측: 2
문제: (가), (나)를 읽은 학생이 <보기>의 임원경제지에 대해 보인 반응으로 적절하지 않은 것은?
정답: 5
GPT-4의 예측: 5


 18%|█▊        | 2/11 [00:35<03:02, 20.32s/it]

...

문제: 발표 내용을 바탕으로 할 때, <보기>에 나타난 학생들의 반응에 대한 이해로 가장 적절한 것은?
정답: 3
GPT-4의 예측: 0
문제: (가)에 활용된 글쓰기 방식으로 가장 적절한 것은?
정답: 4
GPT-4의 예측: 4
문제: <보기>는 (가)의 마지막 문단의 초고이다. <보기>를 고쳐 쓰기 위해 친구들이 조언한 내용 중 반영되지 않은 것은?
정답: 4
GPT-4의 예측: 4
문제: [A], [B]에 대한 설명으로 적절하지 않은 것은?
정답: 3
GPT-4의 예측: 2
문제: (가)와 (나)를 고려할 때, ‘학생 1’이 쓴 [회의록]의 내용 중 적절하지 않은 것은?
정답: 2
GPT-4의 예측: 3


 91%|█████████ | 10/11 [05:00<00:34, 34.52s/it]

문제: 다음은 (나)를 바탕으로 학생들이 만든 지도의 초안이다. ㉠～㉤에 대한 반응으로 가장 적절한 것은?
정답: 4
GPT-4의 예측: 3
문제: 다음은 초고를 작성하기 전에 학생이 떠올린 생각이다. ㉠～㉤ 중, 학생의 초고에 반영되지 않은 것은?
정답: 5
GPT-4의 예측: 0
문제: 다음은 초고를 읽은 교지 편집부 학생의 조언이다. 이를 반영 하여 [A]를 작성한다고 할 때, 가장 적절한 것은?
정답: 2
GPT-4의 예측: 2


100%|██████████| 11/11 [05:18<00:00, 28.96s/it]

문제: <보기>는 초고를 보완하기 위해 추가로 수집한 자료이다. 자료 활용 방안으로 적절하지 않은 것은?
정답: 1
GPT-4의 예측: 1


In [8]:
score_percentage = (earned_score / total_score) * 100
print(f"GPT-4의 점수: {earned_score}/{total_score} ({score_percentage:.2f}%)")

if score_percentage > 50:
    print("통과!")
else:
    print("50점을 넘지 못했습니다.")

GPT-4의 점수: 80/100 (80.00%)
통과!
